In [10]:
# created on Dec 24, 2020
# modified on April 19, 2022
# @author:          Tyson Thoi
# @email:           tpthoi@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [11]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [12]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [23]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "pJkrFK9vAgPuHC40oZ9fB0MmQ"
    consumer_secret = "3LnkkNOphx5g4q8nTQO4UqdGHbnxsQxwiFSJONIrMedmx72zQc"
    access_token = "1516552658814873609-Rq5uEpJ8tThF1tHddg80ATvscCLTyV"
    access_token_secret = "n7QlKIe2UR5MHbXH1qabp9o0F2vRZGw36pwq2bVGhElmr"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-82.5952191051634, 25.155229305240482,-81.01429227865468, 30.45020845765558]

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1516566976621158409, 'username': 'jimrobenalt', 'created_at': 'Tue Apr 19 23:58:38 +0000 2022', 'lng': -83.804475, 'lat': 27.698681999999998, 'text': '@MaryLTrump @natsechobbyist God bless the fighters among us. Let’s all stand with her'}
{'id': 1516566977233530880, 'username': 'aasfxo', 'created_at': 'Tue Apr 19 23:58:38 +0000 2022', 'lng': -82.5628725, 'lat': 27.409205999999998, 'text': 'OG! Never fall for em’ liesss'}
{'id': 1516566987501101061, 'username': 'JimWill74380031', 'created_at': 'Tue Apr 19 23:58:40 +0000 2022', 'lng': -81.0442415, 'lat': 29.291700499999997, 'text': '@Love_Happinezz @GovRonDuhSantis @GovRonDeSantis As a society we have mutually agreed to operates under 1000s of “mandates”.'}
{'id': 1516566993830297613, 'username': 'LordKrono2031', 'created_at': 'Tue Apr 19 23:58:42 +0000 2022', 'lng': -82.3247015, 'lat': 28.2245025, 'text': '@Boabbysam @PammsyB No such thing as liberal repercussions. It is always someone elses fault or a lie.'}
{'id': 1516567004295

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
